In [153]:
import skmob
import pandas as pd
import csv, os, sys, json
import urllib.request
from skmob.preprocessing import clustering


In [164]:
def coords_to_place(x, y):
    client_id = "ktta1uumf3"
    client_secret = "PDFJ27DxqU6bQSsf40GE9DwybUnlgRzZnni42Dqi"
    coords = ','.join([x, y])
    
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=" + \
        coords + "&sourcecrs=epsg:4326&output=json&orders=roadaddr"

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        addr_info = response_body.decode('utf-8')
        addr_info = json.loads(addr_info)
    else:
        print("Error Code:" + rescode)

    if len(addr_info['results']) == 0:
        return "none"

    results = addr_info['results'][0]
    land = results['land']
    building_name = land['addition0']['value']
    road_name = land['name']
    number1 = land['number1']
    number2 = land['number2']

    search = ' '.join([building_name, road_name, number1, number2])

    return search

In [155]:
def place_to_category(place):
    if len(place) == 0 or place == "none":
        return "none"

    client_id = "G3_TXQoFDd0lBFsM8fpG"
    client_secret = "oJaqz0CK0W"
    encText = urllib.parse.quote(place)
    url = "https://openapi.naver.com/v1/search/local?query=" + encText  # JSON 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        category_info = response_body.decode('utf-8')
        category_info = json.loads(category_info)
    else:
        print("Error Code:" + rescode)

    if len(category_info['items']) == 0:
        return "none"
    item = category_info['items'][0]
    category = item['category']

    return category

In [214]:
device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()

for device_id in device_id_list:
    path_dir = './cluster/'+device_id
    file_list = os.listdir(path_dir)

    fields = ['deviceid', 'latitude', 'longitude', 'cluster', 'address']

    for cluster in file_list:
        if not cluster.startswith("cluster_"):
            continue

        result = pd.DataFrame(columns=fields)
        with open('./cluster/'+device_id+'/'+cluster, newline='') as file:
            df = pd.read_csv(file)
            for idx in range (len(df)):
                latitude = df.loc[idx]['latitude']
                longitude = df.loc[idx]['longitude']
                x = "{:.7f}".format(latitude)
                y = "{:.7f}".format(longitude)
                address = coords_to_place(y, x)
                res = pd.DataFrame({
                    'deviceid': [df.loc[idx]['deviceid']],
                    'latitude': [latitude],
                    'longitude': [longitude],
                    'cluster': [df.loc[idx]['cluster']],
                    'address': [address]
                })
                result = pd.concat([result, res])

        c = cluster.split('_')[1].split('.')[0]
        result.to_csv("./cluster/"+device_id+'/address_'+ c +'.csv', index=False)

In [200]:
for device_id in device_id_list:
    path_dir = './cluster/'+device_id
    file_list = os.listdir(path_dir)

    fields = ['deviceid', 'latitude', 'longitude',
              'datetime', 'leaving_datetime', 'cluster', 'category']
    f = open('./cluster/'+device_id+'/category.csv', 'w', newline='')
    write = csv.writer(f)
    write.writerow(fields)

    for cluster in file_list:
        if not cluster.startswith("cluster_"):
            continue
        with open('./cluster/'+device_id+'/'+cluster, newline='') as file:
            df = pd.read_csv(file)
            df['datetime'] = pd.to_datetime(df['datetime'])
            # df['year'] = df['datetime'].map(lambda x: x.isocalendar()[0])
            # df['week'] = df['datetime'].map(lambda x: x.isocalendar()[1])
            latitude = df['latitude'].mean()
            longitude = df['longitude'].mean()

            idx = 0
            dif = 1e6
            for row in range(len(df)):
                tmp = abs(df.loc[row]['latitude']-latitude) + \
                    abs(df.loc[row]['longitude']-longitude)
                if (dif > tmp):
                    idx = row
                    dif = tmp

            x = "{:.7f}".format(df.loc[idx]['latitude'])
            y = "{:.7f}".format(df.loc[idx]['longitude'])
            place = coords_to_place(y, x)
            category = place_to_category(place)

            df['category'] = category
            write.writerow(df.loc[idx])

    f.close()

In [219]:
for device_id in device_id_list:
    df = pd.read_csv('./cluster/'+device_id+'/category.csv')
    df = df.sort_values(by=['cluster'])
    df.to_csv("./cluster/"+device_id+'/category.csv', index=False)